In [14]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import scipy
from scipy.sparse import rand
import scipy.sparse as sparse

import cvxpy as cp

from sklearn.model_selection import ParameterGrid

import copy
from time import time
import random

import warnings
warnings.filterwarnings('ignore')

# Matrix generation

In [17]:
def get_matrix(n, density):
    matr = np.zeros((n,n))
    q_p = np.zeros(n)
    gamma = -1
    vector_s = np.array(scipy.sparse.random(int(n*(n-1)/2), 1, density=density, random_state=15).todense())
    vector_t = np.array(scipy.sparse.random(n, 1, density=density, random_state=16).todense()).flatten()
    
    for i in range(n-1):
        for j in range(i, n):
            if i != j:
                if vector_s[i+j] > 0.000001 or vector_s[i+j] < -0.000001:
                    matr[i][j] = random.uniform(-10, 0)
                    matr[j][i] = matr[i][j]
            if i == j:
                matr[i][j] = random.uniform(-1, 1)
    for i in range(n):
        if vector_t[i] > 0.000001 or vector_t[i] < -0.000001:
            q_p[i] = random.uniform(-1, 0)
    q_pt = q_p.reshape(-1,1)
    M_p = np.vstack([np.hstack([gamma, q_p]).reshape(1,-1), np.hstack([q_pt, matr])])      
    return M_p

# SDP

In [19]:
def mySDP(M0, n, m, density):
    X = cp.Variable((n+1, n+1), symmetric=True)
    
    constraints = [cp.trace(get_matrix(n, density)@X.T) <= 0 for i in range(0, m)]
    constraints += [X>>0]
    constraints += [X[0][0] == 1]
    constraints += [G@cp.diag(X)[1:] <= h]
    
    problem = cp.Problem(cp.Minimize(cp.trace(M0@X.T)), constraints)
    
    start = time() 
    problem.solve(solver = 'MOSEK')
    end = time()
    Time = end - start
  
    print('n =', n, '\t m =', m, '\t density =', density)
    print('Time:', Time)
    print(problem.value)
    return n, m, density, Time

In [7]:
N, M, Density = [100], [100], [0.5]
Results = []

for n in N:
    G = np.random.uniform(0, 1000, size=(n+2, n))
    h = G @ np.random.uniform(0, 1000, size=(n,))
    
    for density in Density:
        M0 = get_matrix(n, density)
        for m in M:
            n, m, density, Time = mySDP(M0, n, m, density)
            Results.append([n, m, density, Time])
to_p = pd.DataFrame(Results, columns = ['n', 'm', 'density', 'Time'])
display(to_p)

n = 100 	 m = 100 	 density = 0.5
Time: 60.60522150993347
-10387480.063083183


,n,m,density,Time
0,100,100,0.5,60.605222


# SOCP

In [20]:
def mySOCP(M0, n, m, density):
    Mp = []
    for i in range(0, m):
        Mp.append(get_matrix(n, density))
        
    X = cp.Variable((n+1, n+1), symmetric=True)
    constraints = [cp.trace(Mp[i]@X.T) <= 0 for i in range(0, m)]
    constraints += [X[0][0] == 1]
    constraints += [G@cp.diag(X)[1:] <= h]
    for j in range(1, n + 1):
        for k in range(0, j):
            if Mp[0][k, j] < -0.000001 or Mp[0][k, j] > 0.000001:
                constraints += [cp.norm(cp.hstack([X[k][k] - X[j][j], 2*X[k][j]])) <= X[k][k] + X[j][j]]
    
    problem = cp.Problem(cp.Minimize(cp.trace(M0@X.T)), constraints)
    
    start = time() 
    problem.solve(solver = 'MOSEK')
    end = time()
    Time = end - start
   
    print('n =', n, '\t m =', m, '\t density =', density)
    print('Time:', Time)
    print(problem.value)
    return n, m, density, Time

In [9]:
N, M, Density = [100], [100], [0.5]
Results = []
for n in N:
    G = np.random.uniform(0, 1000, size=(n+2, n))
    h = G @ np.random.uniform(0, 1000, size=(n,))
    
    for density in Density:
        M0 = get_matrix(n, density)
        for m in M:
            n, m, density, Time = mySOCP(M0, n, m, density)
            Results.append([n, m, density, Time])
to_p = pd.DataFrame(Results, columns = ['n', 'm', 'density', 'Time'])
display(to_p)

n = 100 	 m = 100 	 density = 0.5
Time: 353.2809724807739
-9646399.222769458


,n,m,density,Time
0,100,100,0.5,353.280972


# Main part

In [21]:
def mySDP_id(M0, n, m, density):
    X = cp.Variable((n+1, n+1), PSD=True)
    
    constraints = [cp.trace(Mp[i]@X.T) <= 0 for i in range(0, m)]
    constraints += [X[0][0] == 1]
    constraints += [G@cp.diag(X)[1:] <= h]
    
    problem = cp.Problem(cp.Minimize(cp.trace(M0@X.T)), constraints)
    
    start = time() 
    problem.solve(solver = 'MOSEK')
    end = time()
    Time = end - start
    
    print('n =', n, '\t m =', m, '\t density =', density)
    print('Time:', Time)
    print(problem.value)
    val = problem.value
    return n, m, density, Time, val

In [22]:
def mySOCP_id(M0, n, m, density):
    X = cp.Variable((n+1, n+1), symmetric=True)
    
    constraints = [cp.trace(Mp[i]@X.T) <= 0 for i in range(0, m)]
    constraints += [X[0][0] == 1]
    constraints += [G@cp.diag(X)[1:] <= h]
    
    for j in range(1, n + 1):
        for k in range(0, j):
            if Mp[0][k, j] < -0.0000001 or Mp[0][k, j] > 0.0000001:
                constraints += [cp.norm(cp.vstack([X[k][k] - X[j][j], 2*X[k][j]])) <= X[k][k] + X[j][j]]
    
    problem = cp.Problem(cp.Minimize(cp.trace(M0@X.T)), constraints)
    
    start = time() 
    problem.solve(solver = 'MOSEK')
    end = time()
    Time = end - start
   
    print('n =', n, '\t m =', m, '\t density =', density)
    print('Time:', Time)
    print(problem.value)
    val = problem.value
    return n, m, density, Time, val

In [12]:
N, M, Density = [50, 100], [50], [0.1]
fResults = []
for n in N:
    G = np.random.uniform(0, 100, size=(n+3, n))
    h = G @ np.random.uniform(0, 100, size=(n,))
    
    for density in Density:
        M0 = get_matrix(n, density)
        for m in M:
            Results = []
            Mp = []
            for i in range(m):
                Mp.append(get_matrix(n, density))
            
            n, m, density, Time, val = mySDP_id(M0, n, m, density)
            Results = [n, m, density, Time]
            Results.append(val)
            
            n, m, density, Time, val = mySOCP_id(M0, n, m, density)
            Results.append(Time)
            Results.append(val)
            fResults.append(Results)

to_p = pd.DataFrame(fResults, columns = ['n', 'm', 'density', 'Time_SDP', 'value_SDP', 'Time_SOCP', 'value_SOCP'])
display(to_p)
to_p.to_csv('memorize.csv', index=False)

n = 50 	 m = 50 	 density = 0.1
Time: 1.807164192199707
-28567.588126761653
n = 50 	 m = 50 	 density = 0.1
Time: 0.3102538585662842
-28567.585759113666
n = 100 	 m = 50 	 density = 0.1
Time: 50.20961308479309
-193564.1780348802
n = 100 	 m = 50 	 density = 0.1
Time: 6.866625070571899
-193564.17182150309


,n,m,density,Time_SDP,value_SDP,Time_SOCP,value_SOCP
0,50,50,0.1,1.807164,-28567.588127,0.310254,-28567.585759
1,100,50,0.1,50.209613,-193564.178035,6.866625,-193564.171822


In [25]:
N, M, Density = [50, 100, 200], [50, 100, 200], [0.10, 0.50]
fResults = []

for n in N:
    G = np.random.uniform(0, 100, size=(n+3, n))
    h = G @ np.random.uniform(0, 100, size=(n,))
    
    for density in Density:
        M0 = get_matrix(n, density)
        for m in M:
            Results = []
            Mp = []
            for i in range(m):
                Mp.append(get_matrix(n, density))
            
            n, m, density, Time, val = mySDP_id(M0, n, m, density)
            Results = [n, m, density, Time]
            n, m, density, Time, val = mySOCP_id(M0, n, m, density)
            Results.append(Time)
            fResults.append(Results)

to_p = pd.DataFrame(fResults, columns = ['n', 'm', 'density', 'Time_SDP', 'Time_SOCP'])
display(to_p)
to_p.to_csv('memo_1.csv', index=False)

n = 50 	 m = 50 	 density = 0.1
Time: 1.6864962577819824
-29138.042965772143
n = 50 	 m = 50 	 density = 0.1
Time: 0.30419301986694336
-29138.040625402005
n = 50 	 m = 100 	 density = 0.1
Time: 1.891951084136963
-29138.04296311721
n = 50 	 m = 100 	 density = 0.1
Time: 0.4129006862640381
-29138.04277819633
n = 50 	 m = 200 	 density = 0.1
Time: 2.0754148960113525
-29138.04296830614
n = 50 	 m = 200 	 density = 0.1
Time: 0.5076336860656738
-29138.041881001787
n = 50 	 m = 50 	 density = 0.5
Time: 2.028547525405884
-319433.77339919773
n = 50 	 m = 50 	 density = 0.5
Time: 3.3410661220550537
-319433.7707261221
n = 50 	 m = 100 	 density = 0.5
Time: 3.0827629566192627
-319433.77339516685
n = 50 	 m = 100 	 density = 0.5
Time: 3.7050576210021973
-319433.77335430984
n = 50 	 m = 200 	 density = 0.5
Time: 2.8254218101501465
-319433.77333060757
n = 50 	 m = 200 	 density = 0.5
Time: 4.389230489730835
-319433.7731634389
n = 100 	 m = 50 	 density = 0.1
Time: 47.23959255218506
-174790.3028241603

,n,m,density,Time_SDP,Time_SOCP
0,50,50,0.1,1.686496,0.304193
1,50,100,0.1,1.891951,0.412901
2,50,200,0.1,2.075415,0.507634
3,50,50,0.5,2.028548,3.341066
4,50,100,0.5,3.082763,3.705058
5,50,200,0.5,2.825422,4.389230
6,100,50,0.1,47.239593,6.661169
7,100,100,0.1,50.303397,6.882581
8,100,200,0.1,53.378165,9.656133
9,100,50,0.5,60.910996,321.991411


# 4.2

In [23]:
def get_matrix42(n, density):
    matr = np.zeros((n,n))
    q_p = np.zeros(n)
    gamma = -1
    vector_s = np.array(scipy.sparse.random(int(n*(n-1)/2), 1, density=density, random_state=15).todense())
    vector_t = np.array(scipy.sparse.random(n, 1, density=1, random_state=16).todense()).flatten()
    
    for i in range(n-1):
        for j in range(i, n):
            if i != j:
                if vector_s[i+j] > 0.000001 or vector_s[i+j] < -0.000001:
                    matr[i][j] = random.uniform(-10, 0)
                    matr[j][i] = matr[i][j]
            if i == j:
                matr[i][j] = random.uniform(-1, 1)
    for i in range(n):
        if vector_t[i] > 0.000001 or vector_t[i] < -0.000001:
            q_p[i] = random.uniform(-1, 0)
    q_pt = q_p.reshape(-1,1)
    M_p = np.vstack([np.hstack([gamma, q_p]).reshape(1,-1), np.hstack([q_pt, matr])])      
    return M_p

In [24]:
fResults = []
density = 0

for n, m in zip([50, 50, 50, 100, 100, 100, 200, 200, 200], [50, 100, 200, 50, 100, 200, 50, 100, 200]):
    G = np.random.uniform(0, 100, size=(n+3, n))
    h = G @ np.random.uniform(0, 100, size=(n,))
    M0 = get_matrix(n, density)
    Results = []
    Mp = []
    
    for i in range(m):
        Mp.append(get_matrix42(n, density))

    n, m, density, Time, val = mySDP_id(M0, n, m, density)
    Results = [n, m, density, Time]

    n, m, density, Time, val = mySOCP_id(M0, n, m, density)
    Results.append(Time)
    fResults.append(Results)

to_p = pd.DataFrame(fResults, columns = ['n', 'm', 'density', 'Time_SDP', 'Time_SOCP'])
display(to_p)
to_p.to_csv('memorize_42.csv', index=False)

n = 50 	 m = 50 	 density = 0
Time: 2.0375208854675293
-1178.2387027880575
n = 50 	 m = 50 	 density = 0
Time: 0.2922179698944092
-1178.2384146622246
n = 50 	 m = 100 	 density = 0
Time: 1.9457674026489258
-1086.7229284910786
n = 50 	 m = 100 	 density = 0
Time: 0.3830084800720215
-1086.7229134090267
n = 50 	 m = 200 	 density = 0
Time: 2.1372621059417725
-780.7104962913768
n = 50 	 m = 200 	 density = 0
Time: 0.48968958854675293
-780.7103661453275
n = 100 	 m = 50 	 density = 0
Time: 44.0188250541687
-3360.557132661495
n = 100 	 m = 50 	 density = 0
Time: 1.0292394161224365
-3360.5571467627506
n = 100 	 m = 100 	 density = 0
Time: 52.85547590255737
-3038.9166802530963
n = 100 	 m = 100 	 density = 0
Time: 1.2047703266143799
-3038.916548624439
n = 100 	 m = 200 	 density = 0
Time: 69.53592658042908
-3339.4374562718494
n = 100 	 m = 200 	 density = 0
Time: 1.601757287979126
-3339.4367317649126
n = 200 	 m = 50 	 density = 0
Time: 1610.7547812461853
-7303.401359528239
n = 200 	 m = 50 	 

,n,m,density,Time_SDP,Time_SOCP
0,50,50,0,2.037521,0.292218
1,50,100,0,1.945767,0.383008
2,50,200,0,2.137262,0.489690
3,100,50,0,44.018825,1.029239
4,100,100,0,52.855476,1.204770
5,100,200,0,69.535927,1.601757
6,200,50,0,1610.754781,5.024548
7,200,100,0,1748.214192,5.419473
8,200,200,0,2768.196394,6.808755


The end.